<a href="https://colab.research.google.com/github/fabianapagliuca/LLM-drug-prescription-assistant/blob/main/web_scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scarico bugiardini con Scrapy da sito https://www.farmaciemedici.it/farmaci/bugiardino/elenco-farmaci/a

In [ ]:
#install scrapy
!pip install scrapy
!pip install transformers datasets
!pip install numba
!pip install transformers[torch] --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.2/254.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found e

In [ ]:
#import library
import scrapy
from scrapy.crawler import CrawlerProcess
from datasets import Dataset
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import datasets

In [ ]:
from numba import cuda

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoConfig

Dataframe

In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
import os

class DrugSpider(scrapy.Spider):
    name = "farmaci"
    start_urls = [
        "https://www.farmaciemedici.it/farmaci/bugiardino/elenco-farmaci/a",
        "https://www.farmaciemedici.it/farmaci/bugiardino/elenco-farmaci/b"]

    def parse(self, response):
        for drug in response.css("div.card-title h3 a"):  # Seleziona tutti gli elementi "a" (link) all'interno di h3 dentro <div con la classe (card-title)
            url = response.urljoin(drug.css("::attr(href)").get())  # Combina più link per ottenere quello completo. Utile quando si segue link di pagine correlate
            nome = drug.css("::text").get().strip()  # Estrae il nome, .strip() leva gli spazi
            yield scrapy.Request(url, callback=self.parse_drug, meta={'nome': nome})

    def parse_drug(self, response):
        nome = response.meta['nome']  # Recupera il nome del farmaco passato tramite meta.

        def extract_detail(selector):  # Estrae il testo usando il selettore CSS, unisce le parti di testo e rimuove gli spazi bianchi.
            detail = response.css(selector).getall()
            return ' '.join([d.strip() for d in detail]) if detail else None

        controindicazioni = extract_detail("h3#controindicazioni_farmaco + p::text, h3#controindicazioni_farmaco + p *::text")
        somministrazione = extract_detail("h3#somministrazione_farmaco + p::text, h3#somministrazione_farmaco + p *::text")
        effetti = extract_detail("h3#effetti_farmaco + p::text, h3#effetti_farmaco + p *::text")
        dosaggio = extract_detail("h3#dosaggi_farmaco + p::text, h3#dosaggi_farmaco + p *::text")
        gravidanza = extract_detail("h3#gravidanza_farmaco + p::text, h3#gravidanza_farmaco + p *::text")

        principio_attivo = extract_detail("p span[itemprop='activeIngredient']::text")
        informazioni_aggiuntive = extract_detail("h2 + p::text, h2 + p *::text")
        prezzo = extract_detail("div.col-md-7 > p:contains('prezzo')")

        if prezzo:
            prezzo_text = scrapy.Selector(text=prezzo).css("span[itemprop='cost']::text").get()
            prezzo = f"il prezzo rilevato per la vendita di questo farmaco è di circa {prezzo_text}€ a confezione."



        yield {  # Genero un dizionario con tutte le informazioni raccolte sul farmaco.
            "nome": nome,
            "principio_attivo": principio_attivo,
            "prezzo": prezzo,
            "informazioni_aggiuntive": informazioni_aggiuntive,
            "controindicazioni_farmaco": controindicazioni,
            "somministrazione_farmaco": somministrazione,
            "effetti_farmaco": effetti,
            "dosaggio_farmaco": dosaggio,
            "gravidanza_farmaco": gravidanza
        }

# Configurare e avviare il processo di Scrapy
process = CrawlerProcess(settings={
    'FEED_FORMAT': 'json',  # Formato JSON del dizionario
    'FEED_URI': 'farmaci2.json',  # Specifico il percorso e il nome del file di output
    'LOG_LEVEL': 'INFO',
})

process.crawl(DrugSpider)
process.start()


INFO:scrapy.utils.log:Scrapy 2.11.2 started (bot: scrapybot)
2024-11-04 11:54:16 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-11-04 11:54:16 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Linux-6.1.85+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-11-04 11:54:16 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Crea il DataFrame dal file JSON
with open('farmaci2.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [ ]:
# Conta i valori None in ogni colonna
none_counts = df.isna().sum()

print(none_counts)

nome                          0
principio_attivo              0
prezzo                        0
informazioni_aggiuntive       0
controindicazioni_farmaco    12
somministrazione_farmaco     13
effetti_farmaco              13
dosaggio_farmaco             21
gravidanza_farmaco           15
dtype: int64


In [ ]:
df_cleaned = df.dropna()
# Mostra il numero di righe rimosse
rows_removed = len(df) - len(df_cleaned)
print(f"Numero di righe rimosse: {rows_removed}")

Numero di righe rimosse: 30


In [ ]:
file_path = '/content/drive/My Drive/TESI/dataset_bugfarmaciULTIMO2.csv'
df_cleaned.to_csv(file_path, index=False)

In [ ]:
import pandas as pd
import json

df_new_sorted = df_cleaned.sort_values(by='nome')

print(df_new_sorted)


         nome                                   principio_attivo  \
1        Abba                     Amoxicillina/acido clavulanico   
517   Abesart                                         Irbesartan   
516   Abilify                                       Aripiprazolo   
512  Abiostil  Neomicina/Eucaliptolo/Pino Essenza/Canfora/Men...   
514      Abis                                         Amlodipina   
..        ...                                                ...   
264  Busilvex                                          Busulfano   
260   Busoded                                         Budesonide   
261    Buspar                                          Buspirone   
259  Bydureon                                          Exenatide   
258    Byetta                                          Exenatide   

                                                prezzo  \
1    il prezzo rilevato per la vendita di questo fa...   
517  il prezzo rilevato per la vendita di questo fa...   
516  il p

In [ ]:
file_path = '/content/drive/MyDrive/TESI/dataset_bugfarmaciULTIMO2.csv'

df_new_sorted.to_csv(file_path, index=False)

In [ ]:

print(df_new_sorted['nome'])


1          Abba
517     Abesart
516     Abilify
512    Abiostil
514        Abis
         ...   
264    Busilvex
260     Busoded
261      Buspar
259    Bydureon
258      Byetta
Name: nome, Length: 526, dtype: object


In [ ]:
print(df_new_sorted.iloc[0])

nome                                                                      Abba
principio_attivo                                Amoxicillina/acido clavulanico
prezzo                       il prezzo rilevato per la vendita di questo fa...
informazioni_aggiuntive      ABBA è un antibiotico che funziona uccidendo i...
controindicazioni_farmaco    Il farmaco ABBA è controindicato in casi di ip...
somministrazione_farmaco     La somministrazione del farmaco ABBA varia in ...
effetti_farmaco              Come tutti i medicinali, ABBA può causare effe...
dosaggio_farmaco             Si consiglia sempre di assumere il farmaco ABB...
gravidanza_farmaco           In caso di gravidanza o sospetto di gravidanza...
Name: 1, dtype: object


In [ ]:
file_path = '/content/drive/MyDrive/TESI/dataset_bugfarmaciULTIMO2.csv'
df = pd.read_csv(file_path)

print(df.head())

       nome                                   principio_attivo  \
0      Abba                     Amoxicillina/acido clavulanico   
1   Abesart                                         Irbesartan   
2   Abilify                                       Aripiprazolo   
3  Abiostil  Neomicina/Eucaliptolo/Pino Essenza/Canfora/Men...   
4      Abis                                         Amlodipina   

                                              prezzo  \
0  il prezzo rilevato per la vendita di questo fa...   
1  il prezzo rilevato per la vendita di questo fa...   
2  il prezzo rilevato per la vendita di questo fa...   
3  il prezzo rilevato per la vendita di questo fa...   
4  il prezzo rilevato per la vendita di questo fa...   

                             informazioni_aggiuntive  \
0  ABBA è un antibiotico che funziona uccidendo i...   
1  Trattamento dell'ipertensione essenziale. Trat...   
2  ABILIFY è indicato per il trattamento della sc...   
3  ABIOSTIL unguento nasale è indicato per

In [ ]:
df.iloc[0]

,0
nome,Abba
principio_attivo,Amoxicillina/acido clavulanico
prezzo,il prezzo rilevato per la vendita di questo fa...
informazioni_aggiuntive,ABBA è un antibiotico che funziona uccidendo i...
controindicazioni_farmaco,Il farmaco ABBA è controindicato in casi di ip...
somministrazione_farmaco,La somministrazione del farmaco ABBA varia in ...
effetti_farmaco,"Come tutti i medicinali, ABBA può causare effe..."
dosaggio_farmaco,Si consiglia sempre di assumere il farmaco ABB...
gravidanza_farmaco,In caso di gravidanza o sospetto di gravidanza...
